In [ ]:
#Notebook is running in tmux
import socket
hostname = socket.gethostname()
from VarInfo import GetVarInfo
print(hostname)

In [ ]:
import ROOT
from ROOT import TMVA
from array import array

In [ ]:
#inputFileS = ROOT.TFile("sig_1000.root")
dirNtuple = "root://cmseos.fnal.gov//store/user/rverma/Output/cms-TT-run2/Ntuple_Skim/"
dirFile = "2016/Semilep/JetBase/"
sigFile = "Semilep_JetBase__TstarTstarToTgammaTgluon_M800_2016_Ntuple.root"
inputFileS = ROOT.TFile.Open("%s/%s/%s"%(dirNtuple, dirFile, sigFile))
sig = inputFileS.Get("AnalysisTree")

bkg = ROOT.TChain("AnalysisTree")
bkgList = ["TTGamma_SingleLept", 
           "TTGamma_Dilepton", 
           "TTGamma_Hadronic", 
           "TTGamma_SingleLept_Pt100", 
           "TTGamma_Dilepton_Pt100", 
           "TTGamma_Hadronic_Pt100",
           "TTGamma_SingleLept_Pt200", 
           "TTGamma_Dilepton_Pt200",
           "TTGamma_Hadronic_Pt200"
          ]
for b in bkgList:
    fPath = "%s/%s/Semilep_JetBase__%s_2016_Ntuple.root"%(dirNtuple, dirFile, b)
    bkg.Add(fPath)
print(bkg.GetEntries())

In [ ]:
methodName = "BDTG_mT800"
weightfile = "dataset/weights/TMVA_TT_Classification_" + methodName + ".weights.xml"

ROOT.TMVA.Tools.Instance()
reader = TMVA.Reader("!Color:!Silent")

vars = GetVarInfo()
#Convert strings to variables
for var in vars.keys():
    exec('%s = %s'%(var, array('f',[0])))
    print('%s = %s'%(var, eval(var)))
Weight_lumi = array('f',[0])

for var in vars.keys():
    reader.AddVariable(var, eval(var))
reader.BookMVA(methodName, weightfile)

In [ ]:
#Declare histograms
hSig_disc = ROOT.TH1D("Sig_Disc","Sig_Disc",40,-1,1)
hBkg_disc = ROOT.TH1D("Bkg_Disc","Bkg_Disc",40,-1,1)
for var in vars.keys():
    nBins  = vars[var][1][0]
    xMin   = vars[var][1][1]
    xMax   = vars[var][1][2]
    cut = ["", "_cut"]
    for cut in cut:
        histSig = 'ROOT.TH1D("Sig_%s%s", "Sig_%s%s", %s, %s, %s)'%(var, cut, var, cut, nBins, xMin, xMax)
        histBkg = 'ROOT.TH1D("Bkg_%s%s", "Bkg_%s%s", %s, %s, %s)'%(var, cut, var, cut, nBins, xMin, xMax)
        exec("hSig_%s%s = %s"%(var, cut, histSig))
        exec("hBkg_%s%s = %s"%(var, cut, histBkg))
        print("hBkg_%s%s = %s"%(var, cut, histBkg))

#Fill hists for signal
print("\nRunning for Sig...\n") 
for ievt, e in enumerate(sig):
    eventSel = int(e.Event_pass_presel_mu and ((e.Jet_size>=5 and e.FatJet_size==0) or (e.Jet_size>=2 and e.FatJet_size==1))  and e.Jet_b_size >=1 and e.Photon_size==1 and e.Photon_et[0] > 100)
    if eventSel>0:
        for var in vars.keys():
            exec("%s[0] = e.%s"%(var, var))
            exec("hSig_%s.Fill(e.%s, e.Weight_lumi)"%(var, var))
        disc = reader.EvaluateMVA(methodName)
        hSig_disc.Fill(disc, e.Weight_lumi)
        if disc>0:
            for var in vars.keys():
                exec("hSig_%s_cut.Fill(e.%s, e.Weight_lumi)"%(var, var))
                pass
        if (ievt%100)==0:
            print('Event = %i/%i, Reco_mass_T = %i, MVA output = %s'%(ievt, sig.GetEntries(), Reco_mass_T[0], disc))
            
#Fill hists for background
print("\nRunning for Bkg...\n")    
for ievt, e in enumerate(bkg):
    eventSel = int(e.Event_pass_presel_mu and ((e.Jet_size>=5 and e.FatJet_size==0) or (e.Jet_size>=2 and e.FatJet_size==1))  and e.Jet_b_size >=1 and e.Photon_size==1 and e.Photon_et[0] > 100)
    if eventSel>0:
        for var in vars.keys():
            exec("%s[0] = e.%s"%(var, var))
            exec("hBkg_%s.Fill(e.%s, e.Weight_lumi)"%(var, var))
        disc = reader.EvaluateMVA(methodName)
        hBkg_disc.Fill(disc, e.Weight_lumi)
        if disc>0:
            for var in vars.keys():
                exec("hBkg_%s_cut.Fill(e.%s, e.Weight_lumi)"%(var, var))
        if (ievt%1000)==0:
            print('Event = %i/%i, Reco_mass_T = %i, MVA output = %s'%(ievt, bkg.GetEntries(), Reco_mass_T[0], disc)) 

In [ ]:
def decoHist(hist, xTit, yTit, color):
    hist.GetXaxis().SetTitle(xTit);
    hist.GetYaxis().SetTitle(yTit);
    hist.SetFillColor(color);
    hist.GetXaxis().SetTitle(xTit);
    hist.GetYaxis().SetTitle(yTit)
    hist.GetYaxis().CenterTitle()
    hist.SetLineColor(color)
    hist.SetLineWidth(2)
    hist.GetXaxis().SetTitleOffset(1.0)
    hist.GetYaxis().SetTitleOffset(1.0)
    hist.GetXaxis().SetTitleSize(0.05);
    hist.GetYaxis().SetTitleSize(0.05);
    hist.GetXaxis().SetTitleSize(0.05);
    hist.GetYaxis().SetTitleSize(0.05);

In [ ]:
%jsroot on
c = ROOT.TCanvas()
ROOT.gPad.Draw()
c.Divide(2, 2)
for i, var in enumerate(vars.keys()):    
    c.cd(i+1)
    print(i, var)
    ROOT.gPad.SetLogy(True)
    ROOT.gStyle.SetOptStat(0)
    exec("decoHist(hBkg_%s, \"%s\", \"Events\", 2)"%(var, var))
    exec("decoHist(hSig_%s, \"%s\", \"Events\", 4)"%(var, var))
    exec("hBkg_%s.Draw()"%var)
    exec("hSig_%s.Draw(\"SAME\")"%var)
    leg = ROOT.TLegend(0.75,0.55,0.95,0.88)
    exec("leg.AddEntry(hBkg_%s, \"Background\", \"PEL\")"%var)
    exec("leg.AddEntry(hSig_%s, \"Signal\", \"PEL\")"%var)
    leg.Draw("SAME")

In [ ]:
%jsroot on
c2 = ROOT.TCanvas()
c2.cd()
ROOT.gPad.SetLogy(True)
ROOT.gPad.Draw()
#hSig_disc.Scale(20)
hBkg_disc.SetLineColor(2)
hSig_disc.SetLineWidth(2)
hSig_disc.GetYaxis().SetRangeUser(0, 100)
hSig_disc.Draw()
hBkg_disc.SetLineWidth(2)
hBkg_disc.Draw("SAME")

In [ ]:
%jsroot on
c = ROOT.TCanvas()
ROOT.gPad.Draw()
c.Divide(2, 2)
for i, var in enumerate(vars.keys()):    
    c.cd(i+1)
    print(i, var)
    ROOT.gPad.SetLogy(True)
    ROOT.gStyle.SetOptStat(0)
    exec("decoHist(hBkg_%s_cut, \"%s\", \"Events\", 2)"%(var, var))
    exec("decoHist(hSig_%s_cut, \"%s\", \"Events\", 4)"%(var, var))
    exec("hBkg_%s_cut.Draw()"%var)
    exec("hSig_%s_cut.Draw(\"SAME\")"%var)
    leg = ROOT.TLegend(0.75,0.55,0.95,0.88)
    exec("leg.AddEntry(hBkg_%s_cut, \"Background\", \"PEL\")"%var)
    exec("leg.AddEntry(hSig_%s_cut, \"Signal\", \"PEL\")"%var)
    leg.Draw("SAME")

In [ ]:
outputFile = ROOT.TFile("Disc_Ntuple.root","RECREATE")
CR = "ttyg_Enriched_SR"
def getHistDir(sample, sysType, CR):
    histDir = "%s/%s/%s"%(sample, CR, sysType)
    return histDir

def writeHist(hist, procDir, outputFile):
    outHistDir = getHistDir(procDir, "Base", CR)
    if not outputFile.GetDirectory(outHistDir):
        outputFile.mkdir(outHistDir)
    outputFile.cd(outHistDir)
    ROOT.gDirectory.Delete("%s;*"%(hist.GetName()))
    print "%20s, %10s, %10s"%(hist.GetName(), procDir, round(hist.Integral()))
    #hNew = hist.Rebin(len(newBins)-1, histNewName, newBins) 
    #hNew.Write()
    hist.Write()

writeList = []
writeList.append([hSig_disc, "Sig", "Base"])
writeList.append([hBkg_disc, "Bkg", "Base"])

for var in vars.keys():
    exec("writeList.append([hSig_%s, \"Sig\", \"Base\"])"%var)
    exec("writeList.append([hBkg_%s, \"Bkg\", \"Base\"])"%var)
    exec("writeList.append([hSig_%s_cut, \"Sig\", \"Base\"])"%var)
    exec("writeList.append([hBkg_%s_cut, \"Bkg\", \"Base\"])"%var)

for write in writeList:
    writeHist(write[0], write[1], outputFile)
    if "Bkg" in write[1]:
        writeHist(write[0], "data_obs", outputFile)
#outputFile.ls()
outputFile.Close()